## Replay visualization

Need good ways to visualize the gameplay

In [ ]:
# https://towardsdatascience.com/building-interactive-network-graphs-using-pyvis-5b8e6e25cf64

In [ ]:
def print_replay(replay):
    #[wolf for wolf in stage["werewolves"]]
    #print("Werewolves \tVillagers")
    for stage in replay:
        wolf_votes = [(f'p_{wolf.split("_")[-1]}', stage["votes"][wolf]) for wolf in stage["werewolves"] if wolf in stage["votes"]]
        villager_votes = [(f'p{villager.split("_")[-1]}', stage["votes"][villager]) for villager in stage["villagers"] if villager in stage["votes"]]
        print(f'Wolves \t : {wolf_votes} \t\t Villagers : {villager_votes}')
        #print([f'{wolf.split("_")[-1]} : {stage["votes"]["player"_{wolf.split("_")[-1]}]}]' for wolf in stage["werewolves"]])
        # for wolf in stage['werewolves']:
        #     wid = wolf.split("_")[-1]
        #     pid = f"player_{wid}"
        #     stage["votes"][pid]
        #     print(f'p_{wid} : {stage["votes"][pid]}')
        # print(stage['votes'])

print_replay(replay[4])